In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

# Load your data from a CSV file into a pandas DataFrame
encoded_df = pd.read_csv('APY.csv')

encoded_df.columns = encoded_df.columns.str.strip()

encoded_df.head()


In [ ]:

# Reset the index
encoded_df = encoded_df.reset_index()
encoded_df = encoded_df.dropna() 
# encoded_df['question'] = encoded_df['question'].astype(str).fillna('')
encoded_df['question'] = 'What is the yield for ' + encoded_df['Crop'] + ' in ' + encoded_df['Crop_Year'].astype(str) + ' during ' + encoded_df['Season'] + '?'
encoded_df['question'].head()

split_point = int(round(encoded_df.shape[0] * 0.7, 0))


In [ ]:

training_df = encoded_df.iloc[:split_point]
eval_df = encoded_df.iloc[split_point::]

training = training_df[['question']]
evaluation = eval_df[['question']]

training.tail()

from datasets import Dataset
# Convert the tokenized data to a Hugging Face Dataset
training_set = Dataset.from_pandas(training)
eval_set = Dataset.from_pandas(evaluation)

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('tiiuae/falcon-7b')

# Tokenize the 'question' column
tokenized_training = tokenizer(encoded_df['question'].tolist(), truncation=True, max_length=512)
tokenized_eval = tokenizer(encoded_df['question'].tolist(), truncation=True, max_length=512)

from datasets import Dataset

# Convert the tokenized data to a Hugging Face Dataset
training_dataset = Dataset.from_dict(tokenized_training)
eval_dataset = Dataset.from_dict(tokenized_eval)

# Save the tokenized dataset to disk
tokenized_training.save_to_disk("./training_dataset")
tokenized_eval.save_to_disk("./eval_dataset")


# Save the tokenized dataset to disk
training_dataset.save_to_disk("./training_dataset")
eval_dataset.save_to_disk("./eval_dataset")
